In [ ]:
import json
from pathlib import Path
import re
from typing import List, Dict, Any, Optional


def extract_model_name_from_filename(filename: str) -> str:
    parts = filename.split("-simulation")
    return parts[0]


def count_words(text: str) -> int:
    if not text or not text.strip():
        return 0
    words = re.findall(r"\b\w+\b", text.lower())
    return len(words)


def analyze_jsonl_simulation_responses(
    simulations_dir_path: str,
) -> Optional[List[Dict[str, Any]]]:
    sim_dir = Path(simulations_dir_path)
    if not sim_dir.is_dir():
        print(f"Error: Directory not found at {sim_dir}")
        return None

    all_response_metrics: List[Dict[str, Any]] = []

    simulation_files = list(sim_dir.glob("*.jsonl"))
    if not simulation_files:
        print(f"No .jsonl files found in {sim_dir}.")
        return None

    print(f"Found {len(simulation_files)} .jsonl simulation files to analyze.\n")

    for file_path in simulation_files:
        model_name = extract_model_name_from_filename(file_path.name)
        print(f"Processing file: {file_path.name} (Model: {model_name})...")
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                for i, line in enumerate(f):
                    try:
                        simulation_data = json.loads(line)
                        user_turns = simulation_data.get("simulation", {}).get(
                            "userTurns", []
                        )

                        for turn_idx, turn in enumerate(user_turns):
                            system_response_obj = turn.get("systemResponse")
                            if not system_response_obj:
                                continue

                            utterance = system_response_obj.get("utterance")
                            if utterance is None:
                                continue

                            word_count = count_words(utterance)

                            all_response_metrics.append(
                                {
                                    "file_name": file_path.name,
                                    "model_name": model_name,
                                    "simulation_index_in_file": i,
                                    "turn_index": turn_idx,
                                    "system_response_utterance": utterance,
                                    "word_count": word_count,
                                }
                            )

                    except json.JSONDecodeError as json_e:
                        print(
                            f"  Error decoding JSON on line {i+1} in file {file_path.name}: {json_e}. Skipping line."
                        )
                    except KeyError as ke:
                        print(
                            f"  Missing key in JSON on line {i+1} in file {file_path.name}: {ke}. Skipping."
                        )
                    except Exception as e:
                        print(
                            f"  Unexpected error on line {i+1} in file {file_path.name}: {e}. Skipping."
                        )
            print(f"Finished processing {file_path.name}.\n")
        except Exception as e:
            print(f"Error opening/reading file {file_path.name}: {e}")

    return all_response_metrics

In [ ]:
import pandas as pd

simulations_path = "../simulations"  # Or your specific path
all_metrics = analyze_jsonl_simulation_responses(simulations_path)

if all_metrics:
    df = (
        pd.DataFrame(all_metrics)
        .groupby("model_name")["word_count"]
        .agg(["sum", "mean", "median", "std", "min", "max"])
        .round(2)
    )
    print(df)

Found 6 .jsonl simulation files to analyze.

Processing file: claude-opus-4-simulation.jsonl (Model: claude-opus-4)...
Finished processing claude-opus-4-simulation.jsonl.

Processing file: gemini-2.5-pro-preview-simulation.jsonl (Model: gemini-2.5-pro-preview)...
Finished processing gemini-2.5-pro-preview-simulation.jsonl.

Processing file: gpt-4.1-simulation.jsonl (Model: gpt-4.1)...
Finished processing gpt-4.1-simulation.jsonl.

Processing file: claude-sonnet-4-simulation.jsonl (Model: claude-sonnet-4)...
Finished processing claude-sonnet-4-simulation.jsonl.

Processing file: gpt-4o-simulation.jsonl (Model: gpt-4o)...
Finished processing gpt-4o-simulation.jsonl.

Processing file: gemini-2.5-flash-preview-05-20-simulation.jsonl (Model: gemini-2.5-flash-preview-05-20)...
Finished processing gemini-2.5-flash-preview-05-20-simulation.jsonl.

                                 sum   mean  median   std  min  max
model_name                                                         
claude-opus-